In [4]:
import glob
import numpy as np

from astropy.modeling.models import Gaussian2D
from astropy.modeling.fitting import LevMarLSQFitter

Get the worst-case (smallest) spot size

In [18]:
files = glob.glob('../*results.npz')
smallest_x = np.inf
smallest_y = np.inf
smallest_file_x = ''
smallest_file_y = ''
for file in files:
    data = np.load(file)
    xx = data['xx_img']
    yy = data['yy_img']
    img = np.abs(data['fresnel_img']) ** 2.
    # fit a 2D Gaussian to data and extract fit params
    p_init = Gaussian2D(
        amplitude=img.max(),
        x_mean=0.,
        y_mean=0.
    )
    fit_p = LevMarLSQFitter()
    p = fit_p(p_init, xx, yy, img)
    print(file, p.x_mean, p.y_mean, p.x_fwhm, p.y_fwhm, p.theta)
    if p.x_fwhm < smallest_x:
        smallest_x = p.x_fwhm
        smallest_file_x = file
    if p.x_fwhm < smallest_y:
        smallest_y = p.y_fwhm
        smallest_file_y = file
print(smallest_file_x, smallest_x)
print(smallest_file_y, smallest_y)

..\F1_to_K3_config_3_field_1_results.npz Parameter('x_mean', value=0.021351630812269567) Parameter('y_mean', value=0.005047163122442153) 0.07642846260675831 0.13464328670218667 Parameter('theta', value=0.03759968830939521)
..\F1_to_K3_config_3_field_8_results.npz Parameter('x_mean', value=-0.34280175668180557) Parameter('y_mean', value=-0.012928508283897846) 2.7680776601369656e-38 2.7680776601369656e-38 Parameter('theta', value=0.021122365826883652)
..\K1_to_FLANGE_config_3_field_1_results.npz Parameter('x_mean', value=0.015469944322659523) Parameter('y_mean', value=-0.3644696312299665) 2.7680776601369656e-38 2.7680776601369656e-38 Parameter('theta', value=3.2673048225429926)


..\K1_to_FLANGE_config_3_field_8_results.npz Parameter('x_mean', value=-0.008039823974292672) Parameter('y_mean', value=0.17204880603995332) 0.03853298656612211 0.03445825027527766 Parameter('theta', value=-0.0035562461713523888)
..\K2_to_K1_config_3_field_1_results.npz Parameter('x_mean', value=0.033440865125625906) Parameter('y_mean', value=-0.0014070447803342045) 2.7680776601369656e-38 2.7680776601369656e-38 Parameter('theta', value=-0.1213452452840738)
..\K2_to_K1_config_3_field_8_results.npz Parameter('x_mean', value=-2.3866351571018023) Parameter('y_mean', value=-0.0284913102873105) 2.7680776601369656e-38 2.7680776601369656e-38 Parameter('theta', value=0.10001570736664717)
..\K3_to_K2_config_3_field_1_results.npz Parameter('x_mean', value=0.04192557510659267) Parameter('y_mean', value=0.002590995039604957) 2.7680776601369656e-38 2.7680776601369656e-38 Parameter('theta', value=0.03896245357842139)
..\K3_to_K2_config_3_field_8_results.npz Parameter('x_mean', value=-0.21085309237672